<a href="https://colab.research.google.com/github/hellfer/Kaggle/blob/main/House_Regression_Beginner_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-prices-advanced-regression-techniques:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5407%2F868283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T143523Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D48ed2ab0e4214b1406ae0d6b9191c4d8147ca6034d448110bf1e083314c83ef58cb9a93decb7e9840eedf4264de210323342c85ffb645cef1c81e87e5ad7806b8b5562b6b657969806f91473e30d552281e81ab5b898a620640f7295e038c96d779e80c75e0f357ec1317dfdaa9f8191666952715c73be0b5de5364b9b9afb60572837b4c8e4120802740b3f64a935e88e9857671032bd9e9c4dbd31e34a23a5dfb509d50776fcaf8f6bed723d7440eb017a0d542cbb338a110ead3417de873a67baa74a723ea8a7edf69d51273ed06f820d13dee7b8ac21c8f112c52107da4375d37cee5593b8210349d44c1e5001dc9e233a82f394edb14b5c890740a9d110,ameshousing-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5347562%2F8891534%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T143523Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3b52a1f793b9d11fc912083886174eed12e7ea314b249b810b6a0a54a0868da655c5ef138352d239b697c7584dc80840511aff2b555aa3496ae25e421c7425884761c119d5dce7978c8ebd61d7f43b99a91c7d3074126a2422f214f8a11426920261489b1731acd4a2adfc2a292b6df827ce991280742447587e07df721bfdad26915c6dba7ff3a7bb97a666b42f785799889e224b0a1b6ca009f07b7e40114279129b014a65dfd1ad847e8120bed17ec1bea4b328a42bf4d65e9ec63888d8e4ef1bcab381fc61b37deb15d9dfd6c6af2f9bd942779027d50aebd9a23a3f2ce5716964f5421aa2b99c24cfccfdadd602bdba0c084ac99d35c1b24c454c4529b7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import skew, norm
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

import xgboost as xgb
from catboost import Pool
from catboost import CatBoostRegressor

import missingno as msno

%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/ameshousing-csv/AmesHousing.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.info()

In [ ]:
train.columns = train.columns.str.replace(' ', '')
train = train.drop(['Order','PID'], axis = 1)
train

In [ ]:
train.describe()

In [ ]:
test.info()

In [ ]:
test.drop(['Id'], axis=1, inplace=True)
test

In [ ]:
train_test = pd.concat([train, test], axis=0, ignore_index=True)
train_test

In [ ]:
msno.matrix(train_test)

In [ ]:
numerical_features = train_test.select_dtypes(include = [int, float]).columns.tolist()
categorical_features = train_test.select_dtypes(include = 'object').columns.tolist()

In [ ]:
sns.distplot(train['SalePrice'], kde=True)

In [ ]:
corr = train_test.select_dtypes(include=['number']).corr()
sns.heatmap(corr, cmap='viridis')

In [ ]:
train_test[numerical_features].hist(bins=25, figsize=(15,15))

In [ ]:
selected_features = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(data=train_test[selected_features], size=2)

In [ ]:
sns.regplot(data=train_test, x='GrLivArea', y='SalePrice')

In [ ]:
sns.regplot(data=train_test, x='TotalBsmtSF', y='SalePrice')

In [ ]:
sns.boxplot(x='OverallQual', y='SalePrice', data=train_test)

In [ ]:
sns.boxplot(x='YearBuilt', y='SalePrice', data=train_test)

In [ ]:
train_test.info()

In [ ]:
train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].apply(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)

In [ ]:
train_test['Functional'] = train_test['Functional'].fillna('Typ')
train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
train_test["Alley"] = train_test["Alley"].fillna("None")
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
train_test['Fence'] = train_test['Fence'].fillna("None")
train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")

for col in ('GarageArea', 'GarageCars'):
    train_test[col] = train_test[col].fillna(0)

for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_test[col] = train_test[col].fillna('None')

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_test[col] = train_test[col].fillna('None')

In [ ]:
useless = ['GarageYrBlt', 'YearRemodAdd']
train_test.drop(useless, axis=1, inplace=True)

In [ ]:
missing_value = train_test.isnull().sum()
columns_with_missing = missing_value[missing_value > 0].index.tolist()
columns_with_missing = [col for col in columns_with_missing if train_test[col].isnull().any()]

print('Columns with missing values:', columns_with_missing)

In [ ]:
def impute_knn(df):
    ttn = train_test.select_dtypes(include=[np.number])
    ttc = train_test.select_dtypes(exclude=[np.number])

    cols_nan = ttn.columns[ttn.isna().any()].tolist()
    cols_no_nan = ttn.columns.difference(cols_nan).values

    for col in cols_nan:
        imp_test = ttn[ttn[col].isna()]
        imp_train = ttn.dropna()
        model = KNeighborsRegressor(n_neighbors=5)  # KNR Unsupervised Approach
        knr = model.fit(imp_train[cols_no_nan], imp_train[col])
        ttn.loc[ttn[col].isna(), col] = knr.predict(imp_test[cols_no_nan])

    return pd.concat([ttn, ttc],axis=1)

train_test = impute_knn(train_test)

In [ ]:
objects = []
for i in train_test.columns:
    if train_test[i].dtype == object:
        objects.append(i)

train_test.update(train_test[objects].fillna('None'))
train_test[columns_with_missing].isna().sum()

In [ ]:
train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                               train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

train_test["HighQualSF"] = train_test["GrLivArea"] + train_test["1stFlrSF"] + train_test["2ndFlrSF"] + 0.5 * train_test["GarageArea"] + 0.5 * train_test["TotalBsmtSF"] + 1 * train_test["MasVnrArea"]

train_test["Age"] = pd.to_numeric(train_test["YrSold"]) - pd.to_numeric(train_test["YearBuilt"])

train_test["Renovate"] = pd.to_numeric(train_test["YearRemod/Add"]) - pd.to_numeric(train_test["YearBuilt"])

In [ ]:
train_test_dummy = pd.get_dummies(train_test)
train_test_dummy

In [ ]:
numeric_features = train_test_dummy.dtypes[train_test_dummy.dtypes != object].index
skewed_features = train_test_dummy[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[skewed_features > 0.5]
skew_index = high_skew.index

for i in skew_index:
    train_test_dummy[i] = np.log1p(train_test_dummy[i])

In [ ]:
target = train['SalePrice']
target_log = np.log1p(target)

fig, ax = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('qq-plot & distribution SalePrice', fontsize=15)

sm.qqplot(target_log, stats.t, distargs=(4,), fit=True, line='45', ax=ax[0])
sns.distplot(target_log, kde=True, hist=True, fit=norm, ax=ax[1])
plt.show()

In [ ]:
HighQualSF_log = np.log1p(train_test["HighQualSF"])

fig, ax = plt.subplots(1, 2, figsize= (15, 5))
fig.suptitle("qq-plot & distribution SalePrice ", fontsize=15)

sm.qqplot(HighQualSF_log, stats.t, distargs=(4,), fit=True, line="45", ax=ax[0])
sns.distplot(HighQualSF_log, kde=True, hist=True, fit=norm, ax=ax[1])
plt.show()

train_test["HighQualSF"] = HighQualSF_log

In [ ]:
GrLivArea_log = np.log1p(train_test["GrLivArea"])

fig, ax = plt.subplots(1, 2, figsize= (15,5))
fig.suptitle("qq-plot & distribution SalePrice ", fontsize=15)

sm.qqplot(GrLivArea_log, stats.t, distargs=(4,), fit=True, line="45", ax=ax[0])
sns.distplot(GrLivArea_log, kde=True, hist=True, fit = norm, ax=ax[1])
plt.show()

train_test["GrLivArea"]= GrLivArea_log

In [ ]:
train = train_test_dummy[0:2930]
test = train_test_dummy[2930:]
test.drop('SalePrice', axis=1, inplace=True)

In [ ]:
ytrain = target_log
xtrain = train.drop('SalePrice', axis=1)

X_train, X_val, y_train, y_val = train_test_split(xtrain, ytrain, test_size=0.5, random_state=42 )
X_train, y_train = xtrain, ytrain
X_train

In [ ]:
X_val

In [ ]:
final_model = CatBoostRegressor(random_seed=42)

In [ ]:
final_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

In [ ]:
final_model.get_all_params()

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
final_pred = final_model.predict(X_val)

In [ ]:
final_score = rmse(y_val, final_pred)
final_score

In [ ]:
submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
test_pred = np.expm1(final_model.predict(test))

In [ ]:
submission['SalePrice'] = test_pred
submission

In [ ]:
submission.to_csv("submission.csv", index=False, header=True)